<img src= "https://play-lh.googleusercontent.com/NVEbUI63X1NWrCtIL1nAzn_GMMfR5R8phrcQm5oSvwdE8jwJai0LF6ydboU15Hdo6A" width="1000" height="50"></img>

<h2 style="text-align:center;color:green;font-family:Monospace;">Explore Hotel aspects and Topic Modeling on Reviews.

</h2>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
import seaborn as sns
import matplotlib.pyplot as plt
!pip install chart_studio
import chart_studio.plotly as py 
import plotly.graph_objs as go
import plotly.express as px

from plotly.offline import iplot, init_notebook_mode


from wordcloud import WordCloud 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 <h2 style="text-align:center;color:green;font-family:Monospace;">Data Loading</h2>

In [ ]:
hotel_data = pd.read_csv('/kaggle/input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')

hotel_data.head(10)

In [ ]:
hotel_data.info()


<h2 style="text-align:center;color:green;font-family:Monospace;">Rating Distribution</h2>

In [ ]:

#plt.figure(figsize=(12,12))
#sns.set(style="white") 
#sns.set(style="whitegrid", color_codes=True)
#sns.displot(hotel_data,x='Rating', hue="Rating")
#plt.show()
fig =  px.histogram(hotel_data, x="Rating", color="Rating",
                   title="Hotel Rating Count")
fig.show()

<p style="color:red;font-family:Monospace;">From Above Distribution we can find out that count of high rating hotels are more as compared to low rating hotels.</p>

In [ ]:
hotel_data['Rating'].value_counts().sort_index()


<p style="color:red;font-family:Monospace;">Here we will use spacy for processing and analyzing data in NLP.</p>

In [ ]:
! spacy download en 
! python3 -m spacy download en_core_web_sm

import spacy


 <h2 style="text-align:center;color:green;font-family:Monospace;">Data Processing</h2>

<p style="color:red;font-family:Monospace;">We will use spacy nlp function to tokenizes the text to produce document object.</p>

In [ ]:
nlp = spacy.load('en_core_web_sm')
def text_process(sentence):
    return str(" ".join([word.lemma_ for word in nlp(sentence) if not word.is_stop and not word.is_punct and not word.like_num]))
    


<p style="color:red;font-family:Monospace;">In the above text_process function we are removing stopwords,punctuations ,numbers and also performing Lemmatization.</p>


In [ ]:
reviews = hotel_data['Review'].apply(text_process)
print(reviews)

In [ ]:
doc = [str(x) for x in reviews]
#print(doc)

In [ ]:
df_review = pd.DataFrame(doc,columns=["Reviews"])
df_review.head()

 <h2 style="text-align:center;color:green;font-family:Monospace;">Generate WordCloud</h2>

In [ ]:

def wordCloud_generator(data, title=None):
    wordcloud = WordCloud(width = 1000, height = 1000,
                          background_color ='black',
                          min_font_size = 10
                         ).generate(" ".join(data.values))
    # plot the WordCloud image                        
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=30)
    plt.show() 

<h2 style="text-align:center;color:green;font-family:Monospace;">Hotel Reviews WordCloud </h2>

In [ ]:
wordCloud_generator(df_review['Reviews'],'Most common words')


 <h2 style="text-align:center;color:green;font-family:Monospace;">Generate Frequency Distribution Graph for tokens appearing in Reviews</h2>

In [ ]:

# function to plot most frequent terms
def freq_words(x, terms = 30):
  all_words = ' '.join([text for text in x])
  all_words = all_words.split()

  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  #plt.figure(figsize=(40,12))
  #ax = sns.barplot(data=d, x= "word", y = "count")
  #ax.set(ylabel = 'Count')
  #plt.show()
  fig =  px.histogram(d, x="word",y="count",
                   title="30 Most Frequent Words",
                   color_discrete_sequence=['indianred'])
  fig.show()

 <h2 style="text-align:center;color:green;font-family:Monospace;">Top 30 Most Frequent Words Appearing in Reviews</h2>

In [ ]:
freq_words(df_review['Reviews'])


In [ ]:
ratings_ = [int(x) for x in hotel_data['Rating']]


In [ ]:
df_ratings = pd.DataFrame(ratings_,columns=['Ratings'])
df_ratings.head()

In [ ]:
joined_data = df_review.join(df_ratings)
joined_data.head()

<p style="color:red;font-family:Monospace;">Getting all the reviews having hotel rating greater than 2.</p>


In [ ]:
df_high_ratings = joined_data[joined_data.Ratings >= 3]
df_high_ratings.head()

 <h2 style="text-align:center;color:green;font-family:Monospace;">WordCloud of high rating hotels.</h2>

In [ ]:
wordCloud_generator(df_high_ratings['Reviews'],'Most common words for high ratings hotel')


> <h2 style="text-align:center;color:green;font-family:Monospace;">Frequency Distribution of High Rating Hotels. </h2>

In [ ]:
freq_words(df_high_ratings['Reviews'])


<p style="color:red;font-family:Monospace;">Getting all the reviews having hotel rating less than 3.</p>


In [ ]:
df_low_ratings = joined_data[joined_data.Ratings < 3]
df_low_ratings.head()

<h2 style="text-align:center;color:green;font-family:Monospace;">WordCloud of low rating hotels.</h2>

In [ ]:
wordCloud_generator(df_low_ratings['Reviews'],'Most common words for low ratings hotel')


 <h2 style="text-align:center;color:green;font-family:Monospace;">Frequency Distribution of Low Rating Hotels. </h2>

In [ ]:
freq_words(df_low_ratings['Reviews'])


<h2 style="text-align:center;color:green;font-family:Monospace;">Sentiment Analysis. </h2>

<p style="color:red;font-family:Monospace;">Let’s look at the overall sentiment analysis.For this we will install the textblob library.</p>

In [ ]:
! pip install -U textblob 


In [ ]:
from textblob import TextBlob
joined_data['polarity']=joined_data['Reviews'].apply(lambda x: TextBlob(x).polarity)
joined_data['subjective']=joined_data['Reviews'].apply(lambda x: TextBlob(x).subjectivity)
joined_data.head(10)

<p style="color:red;font-family:Monospace;">We then can calculate the sentiment through the polarity function. polarity ranges from -1 to 1, with -1 being negative and 1 being positive. The TextBlob can also use the subjectivity function to calculate subjectivity, which ranges from 0 to 1, with 0 being objective and 1 being subjective.</p>

<h2 style="text-align:center;color:green;font-family:Monospace;">What's good in high rating hotels. </h2>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

def ngram_generation(data):
    stoplist = stopwords.words('english') + ['hotel']
    c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
    # matrix of ngrams
    ngrams = c_vec.fit_transform(data)
    # count frequency of ngrams
    count_values = ngrams.toarray().sum(axis=0)
    # list of ngrams
    vocab = c_vec.vocabulary_
    df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'frequency', 1:'bigram/trigram'})
    return df_ngram

In [ ]:
high_ratings = ngram_generation(df_high_ratings['Reviews'])

In [ ]:
high_ratings.head(10)

In [ ]:
def ngram_frequency(data):
    d = data.nlargest(columns="frequency", n =20) 
    #plt.figure(figsize=(40,12))
    #ax = sns.barplot(data=d, x= "bigram/trigram", y = "frequency")
    #ax.set(ylabel = 'Count')
    #plt.show()
    fig =  px.histogram(d, x="bigram/trigram",y="frequency",
                   title="Bigram/Trigram Frequency Plot",
                   color_discrete_sequence=['olive'])
    fig.show()

In [ ]:
ngram_frequency(high_ratings)

<h2 style="text-align:center;color:green;font-family:Monospace;">Key Features of 5 star rating hotels. </h2>

In [ ]:
df_rating_5 = joined_data[joined_data.Ratings == 5]
df_rating_5.head()

In [ ]:
ratings_5 = ngram_generation(df_rating_5['Reviews'])

In [ ]:
ratings_5.head(20)

In [ ]:
ngram_frequency(ratings_5)

<p style="color:red;font-family:Monospace;">From above bigram/trigram analysis we can conclude that key facilities available in 5 star rating hotels are :
<ol style="color:red;font-family:Monospace;"> <li>Friendly and Helpful Staff.</li>
    <li> Clean and Large Rooms.</li>
    <li> Minute Walks </li>
    <li> Great Location.</li>


</ol></p>


<h2 style="text-align:center;color:green;font-family:Monospace;">Issues with 1 star rating hotels. </h2>

In [ ]:
df_rating_1 =  joined_data[joined_data.Ratings == 1]
df_rating_1.head(10)

In [ ]:
low_ratings =  ngram_generation(df_rating_1['Reviews'])

In [ ]:
low_ratings.head(20)

In [ ]:
ngram_frequency(low_ratings)

<p style="color:red;font-family:Monospace;">From above bigram/trigram analysis we can conclude that issues with  1 star rating hotels are :
<ol style="color:red;font-family:Monospace;"> <li>Issues Related with air conditioners.</li>
    <li> Credit Card Payments .</li>
    <li> Small Rooms </li>
</ol></p>
